# Решение с помощью pandas

In [46]:
import pandas as pd
import os

## 1. Получение данных

Датасет вот отсюда: http://archive.ics.uci.edu/ml/datasets/Adult

In [47]:
# датасет не включает имена колонок, поэтому задаем их вручную
column_names = [
    'age',
    'workclass',
    'fnlwgt',
    'education',
    'education-num',
    'marital-status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'capital-gain',
    'capital-loss',
    'hours-per-week',
    'native-country',
    'salary'
]

# загружаем из файлов обучающую и тестовую выборки
train_df = pd.read_csv('/home/admin/Документы/!PLEHA/LAB /II_LABS/LABS_II/LABII/DATA_LABS/adult.data', names=column_names)
test_df = pd.read_csv('/home/admin/Документы/!PLEHA/LAB /II_LABS/LABS_II/LABII/DATA_LABS/adult.test', names=column_names)

## 2. Подготовка данных

In [48]:
train_df['native-country'].value_counts()

native-country
United-States                 29170
Mexico                          643
?                               583
Philippines                     198
Germany                         137
Canada                          121
Puerto-Rico                     114
El-Salvador                     106
India                           100
Cuba                             95
England                          90
Jamaica                          81
South                            80
China                            75
Italy                            73
Dominican-Republic               70
Vietnam                          67
Guatemala                        64
Japan                            62
Poland                           60
Columbia                         59
Taiwan                           51
Haiti                            44
Iran                             43
Portugal                         37
Nicaragua                        34
Peru                             31
France       

In [49]:
test_df['native-country'].value_counts()

native-country
United-States                 14662
Mexico                          308
?                               274
Philippines                      97
Puerto-Rico                      70
Germany                          69
Canada                           61
India                            51
El-Salvador                      49
China                            47
Cuba                             43
England                          37
South                            35
Dominican-Republic               33
Italy                            32
Haiti                            31
Portugal                         30
Japan                            30
Poland                           27
Columbia                         26
Jamaica                          25
Guatemala                        24
Greece                           20
Vietnam                          19
Ecuador                          17
Iran                             16
Peru                             15
Nicaragua    

In [50]:
# Удаляем незначимые пробелы во всех строковых данных
train_df = train_df.apply(lambda x: x.str.strip() if x.dtype == 'object' else x)

# Удаляем записи со значением 'Holand-Netherlands' в поле 'native-country'
train_df = train_df[train_df['native-country'] != 'Holand-Netherlands']

# Удаляем незначимые пробелы в строках также и в тестовой выборке
test_df = test_df.apply(lambda x: x.str.strip() if x.dtype == 'object' else x)

# Удаляем строки с пустыми данными для кросс-валидации в тестовой выборке
test_df.dropna(inplace = True)


In [51]:
print(train_df['salary'].value_counts())
print(test_df['salary'].value_counts())

salary
<=50K    24719
>50K      7841
Name: count, dtype: int64
salary
<=50K.    12435
>50K.      3846
Name: count, dtype: int64


In [52]:
print('Размер обучающей выборки: ', train_df.shape)
train_df.head()

Размер обучающей выборки:  (32560, 15)


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [53]:
print('Размер тестовой выборки: ', test_df.shape)
test_df.head()

Размер тестовой выборки:  (16281, 15)


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
1,25,Private,226802.0,11th,7.0,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K.
2,38,Private,89814.0,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K.
3,28,Local-gov,336951.0,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States,>50K.
4,44,Private,160323.0,Some-college,10.0,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688.0,0.0,40.0,United-States,>50K.
5,18,?,103497.0,Some-college,10.0,Never-married,?,Own-child,White,Female,0.0,0.0,30.0,United-States,<=50K.


In [54]:
# проверим, что пропусков нет в обучающей выборке
train_df.isnull().sum()

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
salary            0
dtype: int64

In [55]:
train_df.dtypes

age                int64
workclass         object
fnlwgt             int64
education         object
education-num      int64
marital-status    object
occupation        object
relationship      object
race              object
sex               object
capital-gain       int64
capital-loss       int64
hours-per-week     int64
native-country    object
salary            object
dtype: object

In [56]:
test_df.dtypes

age                object
workclass          object
fnlwgt            float64
education          object
education-num     float64
marital-status     object
occupation         object
relationship       object
race               object
sex                object
capital-gain      float64
capital-loss      float64
hours-per-week    float64
native-country     object
salary             object
dtype: object

In [57]:
# почему-то возраст в тестовой определился как строка, переведем в число
test_df.age = pd.to_numeric(test_df.age)
# также числовые поля, которые в обучающей были целочисленными, здесь стали
# отчего-то вещественными, это может привести к проблемам в дальнейшем
test_df.fnlwgt = test_df.fnlwgt.astype('int64')
test_df['education-num'] = test_df['education-num'].astype('int64')
test_df['capital-gain'] = test_df['capital-gain'].astype('int64')
test_df['capital-loss'] = test_df['capital-loss'].astype('int64')
test_df['hours-per-week'] = test_df['hours-per-week'].astype('int64')

In [58]:
test_df.dtypes

age                int64
workclass         object
fnlwgt             int64
education         object
education-num      int64
marital-status    object
occupation        object
relationship      object
race              object
sex               object
capital-gain       int64
capital-loss       int64
hours-per-week     int64
native-country    object
salary            object
dtype: object

In [59]:
# сохраним результат в csv, чтобы потом можно было легко использовать выборки
train_df.to_csv('train.csv', index=False, header=False)
test_df.to_csv('test.csv', index=False, header=False)

In [60]:
# для категориальных полей выведем количество категорий
train_df.select_dtypes('object').apply(pd.Series.nunique, axis=0)

workclass          9
education         16
marital-status     7
occupation        15
relationship       6
race               5
sex                2
native-country    41
salary             2
dtype: int64

In [61]:
# и для тестовой выборки тоже
test_df.select_dtypes('object').apply(pd.Series.nunique, axis=0)

workclass          9
education         16
marital-status     7
occupation        15
relationship       6
race               5
sex                2
native-country    41
salary             2
dtype: int64

In [62]:
# Преобразуем категориальные переменные в One Hot Encoding
# Предварительно заменим значения в поле 'salary' на более удобные для обработки
train_df['salary'] = train_df['salary'].apply(lambda x: 0 if x == '<=50K' else 1)
test_df['salary'] = test_df['salary'].apply(lambda x: 0 if x == '<=50K.' else 1)

# Преобразуем все категориальные переменные в One Hot Encoding
train_df = pd.get_dummies(train_df)
test_df = pd.get_dummies(test_df)

# Выводим размер признаков в обучающей и тестовой выборках
print('Размер признаков в обучающей выборке: ', train_df.shape)
print('Размер признаков в тестовой выборке: ', test_df.shape)


Размер признаков в обучающей выборке:  (32560, 108)
Размер признаков в тестовой выборке:  (16281, 108)


In [63]:
# train_df, test_df = train_df.align(test_df, join = 'inner', axis = 1)

# print('Размер признаков в обучающей выборке: ', train_df.shape)
# print('Размер признаков в тестовой выборке: ', test_df.shape)

In [64]:
train_df.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,salary,workclass_?,workclass_Federal-gov,workclass_Local-gov,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,39,77516,13,2174,0,40,0,False,False,False,...,False,False,False,False,False,False,False,True,False,False
1,50,83311,13,0,0,13,0,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2,38,215646,9,0,0,40,0,False,False,False,...,False,False,False,False,False,False,False,True,False,False
3,53,234721,7,0,0,40,0,False,False,False,...,False,False,False,False,False,False,False,True,False,False
4,28,338409,13,0,0,40,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [65]:
# определимся с целевым признаком
X_train = train_df.drop('salary', axis=1)
y_train = train_df['salary']

X_test = test_df.drop('salary', axis=1)
y_test = test_df['salary']

In [66]:
# нормализуем данные
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range = (0, 1))
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [67]:
X_test[:5]

array([[0.10958904, 0.14569009, 0.4       , 0.        , 0.        ,
        0.39795918, 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 1.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 1.        , 0.        , 0.        , 0.        ,
        0.        , 1.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

## 3. Анализ данных

In [68]:
# используем модель логистической регрессии для классификации
from sklearn.linear_model import LogisticRegression


In [69]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

LogisticRegression()

In [70]:
# прогноз на первых пяти примерах из тестовой выборки
lr.predict(X_test[:5])

array([0, 0, 0, 1, 0])

In [71]:
lr_pred = lr.predict(X_test)

# для оценки используем метрику accuracy (точность)
from sklearn.metrics import accuracy_score, precision_score, recall_score

print(accuracy_score(y_test, lr_pred))
print(precision_score(y_test, lr_pred))
print(recall_score(y_test, lr_pred))

0.8510533750998096
0.7275056035862952
0.5907436297451898


In [72]:
print(lr_pred)
len(lr_pred)

[0 0 0 ... 1 0 1]


16281

In [73]:
# используем метод k ближайших соседей для классификации
from sklearn.neighbors import KNeighborsClassifier

lr = KNeighborsClassifier(n_neighbors=7)
lr.fit(X_train, y_train)

lr_pred = lr.predict(X_test)

print(accuracy_score(y_test, lr_pred))
print(precision_score(y_test, lr_pred))
print(recall_score(y_test, lr_pred))

0.8288802899084823
0.6568047337278107
0.5772230889235569


## Метрики классификации

Обозначим один из классов позитивным, затем посчитаем из всех предсказанных объектов количество:

TP - количество верно предсказанных позитивных исходов

TN - количество верно предсказанных негативных исходов

FP - количество неверно предсказанных позитивных исходов

FN - количество неверно предсказанных негативных исходов

Метрика accuracy (переводят как процент верно предсказанных) равна $\frac{TP+TN}{TP+TN+FP+FN}.$

Метрика precision (точность) равна $\frac{TP}{TP + FP}.$ Интуитивно ее можно воспринимать как способность алгоритма не называть исход позитивным, если он негативный.

Метрика recall (полнота) равна $\frac{TP}{TP + FN}.$ Интуитивно это способность алгоритма верно угадать все позитивные исходы.

In [74]:
lr_pred_train = lr.predict(X_train)

print(accuracy_score(y_train, lr_pred_train))
print(precision_score(y_train, lr_pred_train))
print(recall_score(y_train, lr_pred_train))

0.8671990171990172
0.7552620119030338
0.6635633210049738


# Решение с помощью PySpark

Сюда необходимо дописать аналогичный код для загрузки и предобработки данных с использованием датафреймов PySpark

In [79]:
!pip install pyspark

In [80]:
from pyspark import SparkConf, SparkContext as sc
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml import Pipeline
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

Для тестирования результат можно запустить код ниже, который обучает модель логистической регрессии.

In [81]:
spark = SparkSession.builder.appName("Logistic-reg-adult").getOrCreate()

In [83]:
# Определение имен колонок
column_names = [
    'age',
    'workclass',
    'fnlwgt',
    'education',
    'education-num',
    'marital-status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'capital-gain',
    'capital-loss',
    'hours-per-week',
    'native-country',
    'salary'
]

# Загрузка данных
train_df = spark.read.csv("/home/admin/Документы/!PLEHA/LAB /II_LABS/LABS_II/LABII/DATA_LABS/adult.data", header=False, inferSchema=True, sep=",").toDF(*column_names)
train_df

DataFrame[age: int, workclass: string, fnlwgt: double, education: string, education-num: double, marital-status: string, occupation: string, relationship: string, race: string, sex: string, capital-gain: double, capital-loss: double, hours-per-week: double, native-country: string, salary: string]

In [84]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType


int_columns = ["fnlwgt", "education-num", "capital-gain", "capital-loss", "hours-per-week"]
for col_name in int_columns:
    train_df = train_df.withColumn(col_name, col(col_name).cast(IntegerType()))

In [85]:
train_df = train_df.filter(train_df["native-country"] != "Holand-Netherlands")

In [87]:
# Identify all string columns
string_columns = [field.name for field in train_df.schema.fields if isinstance(field.dataType, StringType)]

# Create a StringIndexer for each string column
indexers = [StringIndexer(inputCol=col, outputCol=f"{col}_index") for col in string_columns]

# Create and fit the pipeline
pipeline = Pipeline(stages=indexers)
model = pipeline.fit(train_df)
indexed_df = model.transform(train_df)

# Replace original string columns with indexed integer values
indexed_df = indexed_df.select([
    col(c) if c not in string_columns
    else col(f"{c}_index").cast("integer").alias(c)
    for c in train_df.columns
])
indexed_df.show()

+---+---------+------+---------+-------------+--------------+----------+------------+----+---+------------+------------+--------------+--------------+------+
|age|workclass|fnlwgt|education|education-num|marital-status|occupation|relationship|race|sex|capital-gain|capital-loss|hours-per-week|native-country|salary|
+---+---------+------+---------+-------------+--------------+----------+------------+----+---+------------+------------+--------------+--------------+------+
| 39|        4| 77516|        2|           13|             1|         3|           1|   0|  0|        2174|           0|            40|             0|     0|
| 50|        1| 83311|        2|           13|             0|         2|           0|   0|  0|           0|           0|            13|             0|     0|
| 38|        0|215646|        0|            9|             2|         9|           1|   0|  0|           0|           0|            40|             0|     0|
| 53|        0|234721|        5|            7|      

In [88]:
indexed_df

DataFrame[age: int, workclass: int, fnlwgt: int, education: int, education-num: int, marital-status: int, occupation: int, relationship: int, race: int, sex: int, capital-gain: int, capital-loss: int, hours-per-week: int, native-country: int, salary: int]

In [89]:
assembler = VectorAssembler(inputCols=column_names, outputCol="features")

indexed_df = assembler.transform(indexed_df)
# обратим внимание, что используемая здесь для обучения модель логистической регрессии
# взята из библиотеки MLLib

lr = LogisticRegression(featuresCol="features", labelCol="salary")
model = lr.fit(indexed_df)

25/04/30 19:09:16 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
25/04/30 19:09:16 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


In [90]:
pred = model.transform(indexed_df)
pred

DataFrame[age: int, workclass: int, fnlwgt: int, education: int, education-num: int, marital-status: int, occupation: int, relationship: int, race: int, sex: int, capital-gain: int, capital-loss: int, hours-per-week: int, native-country: int, salary: int, features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [91]:
summary = model.summary

In [92]:
summary.accuracy

1.0

In [93]:
summary.precisionByLabel

[1.0, 1.0]

In [94]:
summary.recallByLabel

[1.0, 1.0]

In [95]:
summary.fMeasureByLabel()

[1.0, 1.0]

 PySpark - регрессия

In [98]:
import os
import pandas as pd
import numpy as np

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext

from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import udf, col

from pyspark.ml.regression import LinearRegression
from pyspark.mllib.evaluation import RegressionMetrics

from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, CrossValidatorModel
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.evaluation import RegressionEvaluator

In [99]:
import seaborn as sns
import matplotlib.pyplot as plt

In [101]:
# сделаем настройки визуализации
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_colwidth', 400)

from matplotlib import rcParams
sns.set(context='notebook', style='whitegrid', rc={'figure.figsize': (18,4)})
rcParams['figure.figsize'] = 18,4

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [105]:
# зададим значение сида для воспроизводимости результатов блокнота
rnd_seed=23
np.random.seed=rnd_seed
np.random.set_state=rnd_seed

In [104]:
# создадим сессию Spark на локальной машине с двумя ядрами
spark = SparkSession.builder.master("local[2]").appName("Linear-Regression-California-Housing").getOrCreate()
spark

In [106]:
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=Logistic-reg-adult>

1. Получение данных

In [109]:
HOUSING_DATA = '/home/admin/Документы/!PLEHA/LAB /II_LABS/LABS_II/LABII/cal_housing.data'

In [110]:
# определяем схему данных, соответствующую заголовку файла csv.
schema = StructType([
    StructField("long", FloatType(), nullable=True),
    StructField("lat", FloatType(), nullable=True),
    StructField("medage", FloatType(), nullable=True),
    StructField("totrooms", FloatType(), nullable=True),
    StructField("totbdrms", FloatType(), nullable=True),
    StructField("pop", FloatType(), nullable=True),
    StructField("houshlds", FloatType(), nullable=True),
    StructField("medinc", FloatType(), nullable=True),
    StructField("medhv", FloatType(), nullable=True)]
)

In [111]:
# загружаем данные из файла в датафрейм
housing_df = spark.read.csv(path=HOUSING_DATA, schema=schema).cache()

In [112]:
# вывод пяти строк, но в более красивой форме
housing_df.show(5)

+-------+-----+------+--------+--------+------+--------+------+--------+
|   long|  lat|medage|totrooms|totbdrms|   pop|houshlds|medinc|   medhv|
+-------+-----+------+--------+--------+------+--------+------+--------+
|-122.23|37.88|  41.0|   880.0|   129.0| 322.0|   126.0|8.3252|452600.0|
|-122.22|37.86|  21.0|  7099.0|  1106.0|2401.0|  1138.0|8.3014|358500.0|
|-122.24|37.85|  52.0|  1467.0|   190.0| 496.0|   177.0|7.2574|352100.0|
|-122.25|37.85|  52.0|  1274.0|   235.0| 558.0|   219.0|5.6431|341300.0|
|-122.25|37.85|  52.0|  1627.0|   280.0| 565.0|   259.0|3.8462|342200.0|
+-------+-----+------+--------+--------+------+--------+------+--------+
only showing top 5 rows



. Подготовка данных

In [113]:
housing_df.columns

['long',
 'lat',
 'medage',
 'totrooms',
 'totbdrms',
 'pop',
 'houshlds',
 'medinc',
 'medhv']

In [114]:
housing_df.printSchema()

root
 |-- long: float (nullable = true)
 |-- lat: float (nullable = true)
 |-- medage: float (nullable = true)
 |-- totrooms: float (nullable = true)
 |-- totbdrms: float (nullable = true)
 |-- pop: float (nullable = true)
 |-- houshlds: float (nullable = true)
 |-- medinc: float (nullable = true)
 |-- medhv: float (nullable = true)



In [115]:
housing_df.select('pop','totbdrms').show(10)

+------+--------+
|   pop|totbdrms|
+------+--------+
| 322.0|   129.0|
|2401.0|  1106.0|
| 496.0|   190.0|
| 558.0|   235.0|
| 565.0|   280.0|
| 413.0|   213.0|
|1094.0|   489.0|
|1157.0|   687.0|
|1206.0|   665.0|
|1551.0|   707.0|
+------+--------+
only showing top 10 rows



In [117]:
result_df = housing_df.groupBy("medage").count().sort("medage", ascending=False)
result_df.show(20)

+------+-----+
|medage|count|
+------+-----+
|  52.0| 1273|
|  51.0|   48|
|  50.0|  136|
|  49.0|  134|
|  48.0|  177|
|  47.0|  198|
|  46.0|  245|
|  45.0|  294|
|  44.0|  356|
|  43.0|  353|
|  42.0|  368|
|  41.0|  296|
|  40.0|  304|
|  39.0|  369|
|  38.0|  394|
|  37.0|  537|
|  36.0|  862|
|  35.0|  824|
|  34.0|  689|
|  33.0|  615|
+------+-----+
only showing top 20 rows



In [119]:
result_df.toPandas().plot.bar(x='medage',figsize=(14, 6))

<Axes: xlabel='medage'>

In [120]:
(housing_df.describe().select(
                    "summary",
                    F.round("medage", 4).alias("medage"),
                    F.round("totrooms", 4).alias("totrooms"),
                    F.round("totbdrms", 4).alias("totbdrms"),
                    F.round("pop", 4).alias("pop"),
                    F.round("houshlds", 4).alias("houshlds"),
                    F.round("medinc", 4).alias("medinc"),
                    F.round("medhv", 4).alias("medhv"))
                    .show())

25/04/30 19:21:19 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-------+---------+--------+---------+--------+-------+-----------+
|summary| medage| totrooms|totbdrms|      pop|houshlds| medinc|      medhv|
+-------+-------+---------+--------+---------+--------+-------+-----------+
|  count|20640.0|  20640.0| 20640.0|  20640.0| 20640.0|20640.0|    20640.0|
|   mean|28.6395|2635.7631| 537.898|1425.4767|499.5397| 3.8707|206855.8169|
| stddev|12.5856|2181.6153|421.2479|1132.4621|382.3298| 1.8998|115395.6159|
|    min|    1.0|      2.0|     1.0|      3.0|     1.0| 0.4999|    14999.0|
|    max|   52.0|  39320.0|  6445.0|  35682.0|  6082.0|15.0001|   500001.0|
+-------+-------+---------+--------+---------+--------+-------+-----------+



In [121]:
housing_df = housing_df.withColumn("medhv", col("medhv")/100000)
housing_df.show(2)

+-------+-----+------+--------+--------+------+--------+------+-----+
|   long|  lat|medage|totrooms|totbdrms|   pop|houshlds|medinc|medhv|
+-------+-----+------+--------+--------+------+--------+------+-----+
|-122.23|37.88|  41.0|   880.0|   129.0| 322.0|   126.0|8.3252|4.526|
|-122.22|37.86|  21.0|  7099.0|  1106.0|2401.0|  1138.0|8.3014|3.585|
+-------+-----+------+--------+--------+------+--------+------+-----+
only showing top 2 rows



In [122]:
# Добавление новых колонок в датафрейм
housing_df = (housing_df.withColumn("rmsperhh", F.round(col("totrooms")/col("houshlds"), 2))
                       .withColumn("popperhh", F.round(col("pop")/col("houshlds"), 2))
                       .withColumn("bdrmsperrm", F.round(col("totbdrms")/col("totrooms"), 2)))

housing_df.show(5)

+-------+-----+------+--------+--------+------+--------+------+-----+--------+--------+----------+
|   long|  lat|medage|totrooms|totbdrms|   pop|houshlds|medinc|medhv|rmsperhh|popperhh|bdrmsperrm|
+-------+-----+------+--------+--------+------+--------+------+-----+--------+--------+----------+
|-122.23|37.88|  41.0|   880.0|   129.0| 322.0|   126.0|8.3252|4.526|    6.98|    2.56|      0.15|
|-122.22|37.86|  21.0|  7099.0|  1106.0|2401.0|  1138.0|8.3014|3.585|    6.24|    2.11|      0.16|
|-122.24|37.85|  52.0|  1467.0|   190.0| 496.0|   177.0|7.2574|3.521|    8.29|     2.8|      0.13|
|-122.25|37.85|  52.0|  1274.0|   235.0| 558.0|   219.0|5.6431|3.413|    5.82|    2.55|      0.18|
|-122.25|37.85|  52.0|  1627.0|   280.0| 565.0|   259.0|3.8462|3.422|    6.28|    2.18|      0.17|
+-------+-----+------+--------+--------+------+--------+------+-----+--------+--------+----------+
only showing top 5 rows



In [123]:
housing_df = housing_df.select("medhv",
                              "totbdrms",
                              "pop",
                              "houshlds",
                              "medinc",
                              "rmsperhh",
                              "popperhh",
                              "bdrmsperrm")

featureCols = ["totbdrms", "pop", "houshlds", "medinc", "rmsperhh", "popperhh", "bdrmsperrm"]

Предобработка

In [125]:
assembler = VectorAssembler(inputCols=featureCols, outputCol="features")
assembled_df = assembler.transform(housing_df)
assembled_df.show(10, truncate=False)

+-----+--------+------+--------+------+--------+--------+----------+-------------------------------------------------------+
|medhv|totbdrms|pop   |houshlds|medinc|rmsperhh|popperhh|bdrmsperrm|features                                               |
+-----+--------+------+--------+------+--------+--------+----------+-------------------------------------------------------+
|4.526|129.0   |322.0 |126.0   |8.3252|6.98    |2.56    |0.15      |[129.0,322.0,126.0,8.325200080871582,6.98,2.56,0.15]   |
|3.585|1106.0  |2401.0|1138.0  |8.3014|6.24    |2.11    |0.16      |[1106.0,2401.0,1138.0,8.301400184631348,6.24,2.11,0.16]|
|3.521|190.0   |496.0 |177.0   |7.2574|8.29    |2.8     |0.13      |[190.0,496.0,177.0,7.257400035858154,8.29,2.8,0.13]    |
|3.413|235.0   |558.0 |219.0   |5.6431|5.82    |2.55    |0.18      |[235.0,558.0,219.0,5.643099784851074,5.82,2.55,0.18]   |
|3.422|280.0   |565.0 |259.0   |3.8462|6.28    |2.18    |0.17      |[280.0,565.0,259.0,3.8461999893188477,6.28,2.18,0.17]  |


In [126]:
standardScaler = StandardScaler(inputCol="features", outputCol="features_scaled")
scaled_df = standardScaler.fit(assembled_df).transform(assembled_df)
scaled_df.select("features", "features_scaled").show(10, truncate=False)

+-------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------+
|features                                               |features_scaled                                                                                                                       |
+-------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------+
|[129.0,322.0,126.0,8.325200080871582,6.98,2.56,0.15]   |[0.30623297630686513,0.2843362208866199,0.3295584480852433,4.38209543579743,2.8211223886115664,0.24648542140099877,2.5828740130262697]|
|[1106.0,2401.0,1138.0,8.301400184631348,6.24,2.11,0.16]|[2.6255323394991694,2.1201592122632746,2.9764882057222772,4.36956799913841,2.522034914747303,0.20315790592035446,2.755065613894688]   |
|[190.0,496.0,177.0,7.2574000358581

In [127]:
train_data, test_data = scaled_df.randomSplit([.8,.2], seed=rnd_seed)
train_data.columns

['medhv',
 'totbdrms',
 'pop',
 'houshlds',
 'medinc',
 'rmsperhh',
 'popperhh',
 'bdrmsperrm',
 'features',
 'features_scaled']

АНАЛИЗ ДАННЫХ

In [130]:

lr = (LinearRegression(featuresCol='features_scaled', labelCol="medhv", predictionCol='predmedhv',
                               maxIter=10, regParam=0.3, elasticNetParam=0.8, standardization=False))
linearModel = lr.fit(train_data)
linearModel.coefficients
featureCols
linearModel.intercept

DenseVector([0.0, 0.0, 0.0, 0.5262, 0.0, 0.0, 0.0])

['totbdrms', 'pop', 'houshlds', 'medinc', 'rmsperhh', 'popperhh', 'bdrmsperrm']

1.001383351659608

In [132]:
coeff_df = pd.DataFrame({"Feature": ["Intercept"] + featureCols, "Co-efficients": np.insert(linearModel.coefficients.toArray(), 0, linearModel.intercept)})
coeff_df = coeff_df[["Feature", "Co-efficients"]]

In [135]:
predictions = linearModel.transform(test_data)
predandlabels = predictions.select("predmedhv", "medhv")
predandlabels.show()

+------------------+-------+
|         predmedhv|  medhv|
+------------------+-------+
|1.1498290146733698|0.14999|
| 1.303758290180578|  0.225|
|1.7529725743151858|  0.225|
|1.6094289295308721|  0.269|
|1.5956645319495766|  0.344|
|1.2917940087238424|  0.367|
|1.8668547795751218|  0.375|
|1.3694233397856368|  0.394|
| 1.530553343912004|  0.398|
|1.5715143715849273|  0.409|
|1.3261913014043996|  0.417|
|1.2299232046905868|  0.425|
| 1.484164070093314|  0.425|
|1.3510338217435216|   0.43|
|1.2619110212709583|  0.436|
|1.3350814464678742|   0.44|
|1.4832778453641753|   0.44|
|1.4337313361080326|  0.444|
|1.3691187093204964|  0.445|
|1.4752185586821598|  0.446|
+------------------+-------+
only showing top 20 rows



In [136]:
print("RMSE: {0}".format(linearModel.summary.rootMeanSquaredError))
print("MAE: {0}".format(linearModel.summary.meanAbsoluteError))
print("R2: {0}".format(linearModel.summary.r2))

RMSE: 0.881985204526886
MAE: 0.678289531991799
R2: 0.4180854895364574


In [138]:
evaluator = RegressionEvaluator(predictionCol="predmedhv", labelCol='medhv', metricName='rmse')
print("RMSE: {0}".format(evaluator.evaluate(predandlabels)))
evaluator = RegressionEvaluator(predictionCol="predmedhv", labelCol='medhv', metricName='mae')
print("MAE: {0}".format(evaluator.evaluate(predandlabels)))
evaluator = RegressionEvaluator(predictionCol="predmedhv", labelCol='medhv', metricName='r2')
print("R2: {0}".format(evaluator.evaluate(predandlabels)))

RMSE: 0.8677342923034236
MAE: 0.673609774576463
R2: 0.42517112885139363


In [139]:
spark.stop()